In [3]:
import numpy as np

# ----------------------------------------
# Forward Algorithm
# ----------------------------------------
def forward(O, A, B, pi):
    T = len(O)
    N = len(pi)
    alpha = np.zeros((T, N))

    # Initialization
    alpha[0] = pi * B[:, O[0]]

    # Recursion
    for t in range(1, T):
        for j in range(N):
            alpha[t, j] = np.sum(alpha[t-1] * A[:, j]) * B[j, O[t]]

    # Termination
    return np.sum(alpha[-1])


# ----------------------------------------
# Viterbi Algorithm
# ----------------------------------------
def viterbi(O, A, B, pi, states):
    T = len(O)
    N = len(pi)

    delta = np.zeros((T, N))
    psi = np.zeros((T, N), dtype=int)

    # Initialization
    delta[0] = pi * B[:, O[0]]

    # Recursion
    for t in range(1, T):
        for j in range(N):
            seq_probs = delta[t-1] * A[:, j]
            psi[t, j] = np.argmax(seq_probs)
            delta[t, j] = np.max(seq_probs) * B[j, O[t]]

    # Backtracking
    path = np.zeros(T, dtype=int)
    path[-1] = np.argmax(delta[-1])

    for t in reversed(range(1, T)):
        path[t-1] = psi[t, path[t]]

    # Convert state indices to names
    decoded_states = [states[i] for i in path]
    return decoded_states, np.max(delta[-1])


# ------------------------------------------
# States and observations
# ------------------------------------------
states = ["Quiet", "Busy"]
observations = ["Low", "Medium", "High"]

# Map names to indices for easy array access
S = {name: i for i, name in enumerate(states)}
O = {name: i for i, name in enumerate(observations)}

# ------------------------------------------
# HMM parameters from Ice-Cream Parlour
# ------------------------------------------
A = np.array([
    [0.2, 0.8],   # Quiet -> Quiet, Busy
    [0.7, 0.3]    # Busy  -> Quiet, Busy
])

B = np.array([
    [0.6, 0.3, 0.1],  # Quiet emits Low, Medium, High
    [0.2, 0.3, 0.5]   # Busy  emits Low, Medium, High
])

pi = np.array([0.7, 0.3])  # Initial probabilities: Quiet, Busy

# ------------------------------------------
# Observed sales sequence: Low, High, High, Medium
# ------------------------------------------
obs_seq = ["Low", "High", "High", "Medium"]
O_idx = [O[x] for x in obs_seq]

# ------------------------------------------
# Example hidden path: Quiet, Busy, Busy, Quiet
# ------------------------------------------
hidden_path = ["Quiet", "Busy", "Busy", "Quiet"]
S_idx = [S[s] for s in hidden_path]

# ------------------------------------------
# Compute joint probability P(S, O)
# ------------------------------------------
def joint_probability(S_idx, O_idx, pi, A, B):
    prob = pi[S_idx[0]] * B[S_idx[0], O_idx[0]]
    for t in range(1, len(S_idx)):
        prev_s = S_idx[t-1]
        curr_s = S_idx[t]
        o = O_idx[t]
        prob *= A[prev_s, curr_s] * B[curr_s, o]
    return prob

P = joint_probability(S_idx, O_idx, pi, A, B)
print("Joint probability P(S, O) =", P)

# ------------------------------------------
# Forward Algorithm: compute P(O | λ)
# ------------------------------------------
P_forward = forward(O_idx, A, B, pi)
print("Forward Algorithm P(O | λ) =", P_forward)

# ------------------------------------------
# Viterbi Algorithm: most probable hidden sequence
# ------------------------------------------
decoded_states, P_viterbi = viterbi(O_idx, A, B, pi, states)
print("Viterbi decoded hidden states:", decoded_states)
print("Viterbi path probability:", P_viterbi)



Joint probability P(S, O) = 0.005292
Forward Algorithm P(O | λ) = 0.0132696
Viterbi decoded hidden states: ['Quiet', 'Busy', 'Busy', 'Quiet']
Viterbi path probability: 0.005292
